In [1]:
%pip install openai-whisper
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sounddevice scipy


  Using cached scipy-1.16.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (61 kB)
Using cached scipy-1.16.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pyaudio


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!apt-get update
!apt-get install portaudio19-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,266 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,963 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main a

### Record Audio in Colab

You can use the `google.colab.output` library to record audio directly in your browser.

After running the code above and recording your audio, the `recorded_audio.wav` file will be saved in your Colab environment. You can then use this file with the Whisper model for transcription.

In [14]:
# from google.colab import output
# from base64 import b64decode
# import tempfile
# import whisper

# # ⚙️ Step 3: Load Whisper model (you can use "base", "small", "medium", or "large")
# whisper_model = whisper.load_model("large")  # You can change to "large" if needed

# # ⚙️ Step 4: JavaScript to record audio in browser
# RECORD = """
# const sleep = time => new Promise(resolve => setTimeout(resolve, time));
# const b2text = blob => new Promise(resolve => {
#   const reader = new FileReader();
#   reader.onloadend = () => resolve(reader.result);
#   reader.readAsDataURL(blob);
# });

# async function record(duration=5000){
#   const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
#   const recorder = new MediaRecorder(stream);
#   let data = [];

#   recorder.ondataavailable = event => data.push(event.data);
#   recorder.start();
#   await sleep(duration);
#   recorder.stop();

#   // Wait for recording to stop
#   await new Promise(resolve => recorder.onstop = resolve);

#   //  Stop all tracks to release the microphone
#   stream.getTracks().forEach(track => track.stop());

#   const blob = new Blob(data, { type: 'audio/webm' });
#   const base64 = await b2text(blob);
#   return base64;
# }
# record();
# """


# Step 5: Python function to record & transcribe
def record_and_transcribe():
    import time
    print(" Get ready... (starting in 2 seconds)")
    time.sleep(2)
    print(" Speak now (for 5 seconds)...")

    audio_data = output.eval_js(RECORD)
    print(" Recording complete, transcribing...")

    audio_bytes = b64decode(audio_data.split(',')[1])

    with tempfile.NamedTemporaryFile(suffix=".webm", delete=False) as tmp_webm:
        tmp_webm.write(audio_bytes)
        tmp_webm.flush()

        result = whisper_model.transcribe(tmp_webm.name, language="en")
        transcription = result["text"].strip()
        print(f"🗣️ Transcription: {transcription}")
        return transcription


In [2]:
!pip install -q transformers torch


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load TinyBERT model (general-purpose, uncased)
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
bert_model = AutoModel.from_pretrained("prajjwal1/bert-tiny")


/home/shubh/Documents/voice_command_project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        # Use the [CLS] token's embedding as sentence representation
        return outputs.last_hidden_state[:, 0, :]  # Shape: [1, hidden_size]


In [5]:
import os
import platform
import datetime

def open_browser():
    if platform.system() == "Windows":
        os.system("start chrome")
    else:  # Linux
        os.system("xdg-open https://www.google.com")

def play_music():
    if platform.system() == "Windows":
        os.system("start wmplayer")  # You can also specify a file path
    else:  # Linux
        # Replace with your music file path or player
        os.system("xdg-open /path/to/your/music.mp3")

def get_time():
    print("Current time:", datetime.datetime.now().strftime("%H:%M:%S"))

def open_notepad():
    if platform.system() == "Windows":
        os.system("start notepad")
    else:  # Linux
        # Try gedit, nano, or any available editor
        try:
            os.system("gedit &")
        except:
            os.system("nano")

def shutdown():
    confirm = input("Are you sure you want to shut down? (yes/no): ")
    if confirm.lower() == "yes":
        if platform.system() == "Windows":
            os.system("shutdown /s /t 1")
        else:  # Linux
            os.system("shutdown now")

In [9]:
commands = [
    "open browser",
    "play music",
    "what time is it",
    "shutdown the system",
    "open notepad"
]
intent_actions = {
    "open_browser": open_browser,
    "play_music": play_music,
    "get_time": get_time,
    "shutdown": shutdown,
    "open_notepad": open_notepad
}


In [6]:
def classify_intent(text):
    # Dummy implementation
    if "browser" in text:
        return "open_browser", 0.95
    elif "music" in text:
        return "play_music", 0.92
    elif "time" in text:
        return "get_time", 0.90
    elif "shutdown" in text:
        return "shutdown", 0.88
    elif "notepad" in text:
        return "open_notepad", 0.91
    else:
        return "invalid_command", 0.50


In [10]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        return outputs.last_hidden_state[:, 0, :]  # [CLS] token

# Compute and cache embeddings for known intents (keys of intent_actions)
intents = list(intent_actions.keys())
intent_embeddings = torch.cat([get_embedding(intent) for intent in intents])

# Classify input using cosine similarity
def classify_intent(user_text, threshold=0.7):
    user_emb = get_embedding(user_text)
    similarity = F.cosine_similarity(user_emb, intent_embeddings)
    best_score = torch.max(similarity).item()
    best_idx = torch.argmax(similarity).item()

    if best_score >= threshold:
        return intents[best_idx], best_score
    else:
        return "invalid_command", best_score

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
import sounddevice as sd
from scipy.io.wavfile import write
import whisper

def record_and_transcribe(duration=5, fs=16000, filename="recorded_audio.wav"):
    print(" Get ready... (starting in 2 seconds)")
    import time
    time.sleep(2)
    print(f" Speak now (for {duration} seconds)...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    write(filename, fs, recording)
    print(" Recording complete, transcribing...")

    whisper_model = whisper.load_model("base")  # or "small", "medium", "large"
    result = whisper_model.transcribe(filename, language="en")
    transcription = result["text"].strip()
    print(f"🗣️ Transcription: {transcription}")
    return transcription

In [12]:
# Compute and cache embeddings for command phrases (not intent keys)
commands = [
    "open browser",
    "play music",
    "what time is it",
    "shutdown the system",
    "open notepad"
]
intents = list(intent_actions.keys())
command_to_intent = dict(zip(commands, intents))

command_embeddings = torch.cat([get_embedding(cmd) for cmd in commands])

# Classify input using cosine similarity
def classify_intent(user_text, threshold=0.7):
    user_emb = get_embedding(user_text)
    similarity = F.cosine_similarity(user_emb, command_embeddings)
    best_score = torch.max(similarity).item()
    best_idx = torch.argmax(similarity).item()

    if best_score >= threshold:
        intent = command_to_intent[commands[best_idx]]
        return intent, best_score
    else:
        return "invalid_command", best_score

In [ ]:
text_input = record_and_transcribe()

intent, score = classify_intent(text_input)

if intent == "invalid_command":
    print(" This command is invalid or not available.")
else:
    print(f" Predicted Intent: {intent} (score: {score:.2f})")

    # Execute the mapped function
    action = intent_actions.get(intent)
    if action:
        print(" Executing action...")
        action()
    else:
        print(" No action mapped for this intent.")

 Get ready... (starting in 2 seconds)
 Speak now (for 5 seconds)...
 Recording complete, transcribing...
🗣️ Transcription: Open browser
 Predicted Intent: open_browser (score: 1.00)
 Executing action...


[7201:7201:0717/062547.160238:ERROR:network_service_instance_impl.cc(612)] Network service crashed, restarting service.
[7201:7201:0717/062604.397954:ERROR:interface_endpoint_client.cc(725)] Message 1759034895 rejected by interface blink.mojom.FrameWidgetHost
